# "2020 March One Day Holding Long Short Portfolio BackTest"
> "2020 March Portfolio"
- toc: false
- branch: master
- badges: false
- comments: true
- categories: [long, short, longshort]
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

In [16]:
#hide_input
import numpy as np
import pandas as pd
import seaborn as sns
import os
import time
import gc
import glob
import matplotlib.pyplot as plt

In [17]:
#hide_input
#df_testp=pd.read_csv('D:\\Downloads\\20194dayreturnpred2019train60K.csv')
#df_test=pd.read_csv('D:\\Downloads\\sample28K20194dayreturnpred2019train60K.csv')
df_tes=pd.read_csv('D:\\Downloads\\10Q20204dayreturnpred2019train60K.csv')

In [18]:
#hide
#df_testp=df_testp.loc[df_testp.label!='nan'].loc[df_testp.Date>=20190101].loc[df_testp.Date<20190201].sort_values(by='Date',ascending=True).copy()
#df_test=df_test.loc[df_test.label!='nan'].loc[df_test.Date>=20190101].loc[df_test.Date<20190201].sort_values(by='Date',ascending=True).copy()
df_tes=df_tes.loc[df_tes.label!='nan'].loc[df_tes.Date>=20200301].loc[df_tes.Date<20200401].sort_values(by='Date',ascending=True).copy()
#df_testp.reset_index(drop=True, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


**Long-Short Portfolio**

In [19]:
#hide
#df_agg=df_testp[['Date','pred','labelr1']].groupby(['Date','pred']).sum().reset_index();
#df_agg['cumreturn1']=0.0
#for l in range(len(df_agg)):
#    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']*df_agg['pred']/100))[:l+1])
    
#hide
#df_ag=df_test[['Date','pred','labelr1']].groupby(['Date','pred']).sum().reset_index();
#df_ag['cumreturn1a']=0.0
#for l in range(len(df_ag)):
#    df_ag['cumreturn1a'][l]=np.prod((1+(df_ag['labelr1']*df_ag['pred']/100))[:l+1])
    
df_a=df_tes[['Date','pred','labelr1']].groupby(['Date','pred']).sum().reset_index();
df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a['cumreturn1b'][l]=np.prod((1+(df_a['labelr1']*df_a['pred']/100))[:l+1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [20]:
#hide
if len(df_a)>0:
    import altair as alt
    import datetime
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_a.Date.min()].loc[df_dow.Date<=df_a.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_a=pd.merge(df_a,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_a.Date.min()].loc[df_sp500.Date<=df_a.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_a=pd.merge(df_a,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    #df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    #df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    #df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    #df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [21]:
#hide_input
df_a

,Date,pred,labelr1,cumreturn1b,cumreturndow,cumreturnsp500
0,2020-03-03,1.0,9.809242,1.098092,1.042203,1.045276
1,2020-03-04,-1.0,-18.958980,1.306280,1.006849,1.007866
2,2020-03-04,1.0,-3.801521,1.256621,1.006849,1.007866
3,2020-03-05,1.0,4.614422,1.314607,0.989678,0.997969
4,2020-03-06,-1.0,0.727056,1.305049,0.914493,0.920270
5,2020-03-06,1.0,-30.402657,0.908279,0.914493,0.920270
6,2020-03-09,-1.0,17.058959,0.753336,0.959665,0.965303
7,2020-03-09,1.0,-2.252747,0.736366,0.959665,0.965303
8,2020-03-10,-1.0,-20.451694,0.886965,0.912768,0.908780
9,2020-03-10,1.0,0.130435,0.888122,0.912768,0.908780


In [22]:
#hide_input
import altair as alt
a1 = alt.Chart(df_a).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1b:Q',
    size='cumreturn1b',
    
).properties(
    width=1000,
    height=1000,
    title='10QPortfolio (blue) vs  Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_a).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_a).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

#a4 = alt.Chart(df_agg).mark_line(color='brown').encode(
#    x='Date:T',
#    y='cumreturn1a:Q',

#).properties(
#    width=1000,
#    height=1000
#)

#a5 = alt.Chart(df_agg).mark_line(color='yellow').encode(
#    x='Date:T',
#    y='cumreturn1b:Q',

#).properties(
#    width=1000,
#    height=1000
#)

a1 + a2 + a3# + a4 + a5

alt.LayerChart(...)

**Long portfolio**

In [23]:
#hide
df_a=df_tes[['Date','pred','labelr1']].loc[df_tes.pred==1.0].groupby(['Date','pred']).sum().reset_index();
df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a['cumreturn1b'][l]=np.prod((1+(df_a['labelr1']*df_a['pred']/100))[:l+1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [24]:
#hide
if len(df_a)>0:
    import altair as alt
    import datetime
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_a.Date.min()].loc[df_dow.Date<=df_a.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_a=pd.merge(df_a,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_a.Date.min()].loc[df_sp500.Date<=df_a.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_a=pd.merge(df_a,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    #df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    #df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    #df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    #df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [25]:
#hide_input
df_a

,Date,pred,labelr1,cumreturn1b,cumreturndow,cumreturnsp500
0,2020-03-03,1.0,9.809242,1.098092,1.042203,1.045276
1,2020-03-04,1.0,-3.801521,1.056348,1.006849,1.007866
2,2020-03-05,1.0,4.614422,1.105093,0.989678,0.997969
3,2020-03-06,1.0,-30.402657,0.769115,0.914493,0.920270
4,2020-03-09,1.0,-2.252747,0.751789,0.959665,0.965303
5,2020-03-10,1.0,0.130435,0.752769,0.912768,0.908780
6,2020-03-11,1.0,-10.558846,0.673286,0.825952,0.818007
7,2020-03-13,1.0,-1.574803,0.662683,0.794484,0.778956
8,2020-03-17,1.0,-1.496719,0.652764,0.798470,0.767782
9,2020-03-19,1.0,-5.400631,0.617511,0.767445,0.739811


In [26]:
#hide_input
import altair as alt
a1 = alt.Chart(df_a).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1b:Q',
    size='cumreturn1b',
    
).properties(
    width=1000,
    height=1000,
    title='10QPortfolio (blue) vs  Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_a).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_a).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

#a4 = alt.Chart(df_agg).mark_line(color='brown').encode(
#    x='Date:T',
#    y='cumreturn1a:Q',

#).properties(
#    width=1000,
#    height=1000
#)

#a5 = alt.Chart(df_agg).mark_line(color='yellow').encode(
#    x='Date:T',
#    y='cumreturn1b:Q',

#).properties(
#    width=1000,
#    height=1000
#)

a1 + a2 + a3# + a4 + a5

alt.LayerChart(...)

**Short Portfolio**

In [27]:
#hide
    
df_a=df_tes[['Date','pred','labelr1']].loc[df_tes.pred==-1.0].groupby(['Date','pred']).sum().reset_index();
df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a['cumreturn1b'][l]=np.prod((1+(df_a['labelr1']*df_a['pred']/100))[:l+1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [28]:
#hide
if len(df_a)>0:
    import altair as alt
    import datetime
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_a.Date.min()].loc[df_dow.Date<=df_a.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_a=pd.merge(df_a,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_a.Date.min()].loc[df_sp500.Date<=df_a.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_a=pd.merge(df_a,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    #df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    #df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    #df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    #df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [29]:
#hide_input
df_a

,Date,pred,labelr1,cumreturn1b,cumreturndow,cumreturnsp500
0,2020-03-04,-1.0,-18.958980,1.189590,0.966078,0.964210
1,2020-03-06,-1.0,0.727056,1.180941,0.877462,0.880408
2,2020-03-09,-1.0,17.058959,0.979485,0.920805,0.923491
3,2020-03-10,-1.0,-20.451694,1.179806,0.875807,0.869416
4,2020-03-11,-1.0,-27.770390,1.507442,0.792506,0.782575
5,2020-03-12,-1.0,-10.973214,1.672857,0.866107,0.855847
6,2020-03-13,-1.0,-22.777778,2.053897,0.762313,0.745215
7,2020-03-16,-1.0,-36.507182,2.803717,0.808017,0.783932
8,2020-03-18,-1.0,0.000000,2.803717,0.769744,0.741475
9,2020-03-20,-1.0,3.725490,2.699265,0.714797,0.686281


In [30]:
#hide_input
import altair as alt
a1 = alt.Chart(df_a).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1b:Q',
    size='cumreturn1b',
    
).properties(
    width=1000,
    height=1000,
    title='10QPortfolio (blue) vs  Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_a).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_a).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

#a4 = alt.Chart(df_agg).mark_line(color='brown').encode(
#    x='Date:T',
#    y='cumreturn1a:Q',

#).properties(
#    width=1000,
#    height=1000
#)

#a5 = alt.Chart(df_agg).mark_line(color='yellow').encode(
#    x='Date:T',
#    y='cumreturn1b:Q',

#).properties(
#    width=1000,
#    height=1000
#)

a1 + a2 + a3# + a4 + a5

alt.LayerChart(...)